# `relatio` for analysis of narratives in the survey data
**Runtime $\sim$ 10min**

----------------------------

This code is based on a short demo of the package `relatio` publicly available from the authors of the relatio method. I use the the main wrapper functions to quickly obtain narrative statements from the corpus of my survey responses. For further details, please refer to the paper: ["Text Semantics Capture Political and Economic Narratives"](https://arxiv.org/abs/2108.01720) 

Notes on inputs and outputs:
It takes as input a text corpus and outputs a list of narrative statements. The pipeline is unsupervised: the user does not need to specify narratives beforehand. Narrative statements are defined as tuples of semantic roles with a (agent, verb, patient, attribute) structure. 

----------------------------

In [ ]:
#import data
import pandas as pd

survey_data_filtered_file = '/Users/emilyrobitschek/git/ETH/SPUR/densification_project/datasets/spur_survey_response_filtered_df.txt'
survey_df = pd.read_csv(survey_data_filtered_file, sep='\t')

subset_col = 'Q12.6_corrected'
df = survey_df[['ResponseId', subset_col]]
df = df.rename(columns={'ResponseId': 'id', subset_col : 'doc'})
df.head()

In [ ]:
#import stop words from ntlk (less extensive list):
import nltk
from nltk.corpus import stopwords
nltk_stops = stopwords.words('english')
nltk_stops.extend(['u'])
print(nltk_stops)
nltk_stops = set(nltk_stops)

## Step 1: Split into sentences

----------------------------

For any new corpus, the first thing you will want to do is to split the corpus into sentences.

We do this on the first 100 tweets. 

The output is two lists: one with an index for the document and one with the resulting split sentences.

----------------------------


In [ ]:
from relatio.utils import split_into_sentences

split_sentences = split_into_sentences(
    df, progress_bar=True
)

for i in range(5):
    print('Document id: %s' %split_sentences[0][i])
    print('Sentence: %s \n' %split_sentences[1][i])

## Step 2: Annotate semantic roles

----------------------------

Once the corpus is split into sentences. You can feed it to the semantic role labeler.

The output is a list of json objects which contain the semantic role annotations for each sentence in the corpus.

----------------------------


In [ ]:
# Note that SRL is time-consuming, in particular on CPUs.
# To speed up the annotation, you can also use GPUs via the "cuda_device" argument of the "run_srl()" function. 

from relatio.wrappers import run_srl

srl_res = run_srl(
    path="https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz", # pre-trained model
    sentences=split_sentences[1],
    progress_bar=True,
)

In [ ]:
# An example of SRL output
srl_res[6]

## Step 3: Build the narrative model

----------------------------

We are now ready to build a narrative model.

The function `build_narrative_model` takes as input the split sentences the SRL annotations for the corpus. It builds a model of low-dimensional narrative statements which may then be used to obtain narrative statements  "out-of-sample".

The function has sensible defaults for most arguments, but the user should at least specify:
- the number of latent unnamed entities to recover (`n_clusters`)
- the embeddings to be used (see here for further details) 

We specify 100 unnamed entities to uncover. The embeddings used are pre-trained glove embeddings.

To speed up the model's training, we also focus on the top 100 most frequent named entities (the default is to mine all named entities).

To improve interpretability, we remove common uninformative words in the corpus (the "stopwords"), as well as one-letter words.

----------------------------

In [ ]:
# NB: This step usually takes several minutes to run. You might want to grab a coffee.
 
from relatio.wrappers import build_narrative_model

narrative_model = build_narrative_model(
    srl_res=srl_res,
    sentences=split_sentences[1],
    embeddings_type="gensim_keyed_vectors",
    embeddings_path="glove-wiki-gigaword-300",
    n_clusters=[[50]], 
    stop_words = nltk_stops,
    progress_bar=True,
)

In [ ]:
# The narrative model is simply a dictionary containing the narrative model's specifics.
print(narrative_model.keys())
print(nltk_stops)

In [ ]:
# Most common named entities
narrative_model['entities'].most_common()[:20]

In [ ]:
# The unnamed entities uncovered in the corpus 
# (automatically labeled by the most frequent phrase in the cluster)
narrative_model['cluster_labels_most_freq']

----------------------------

In practice, `build_narrative_model` is a flexible wrapper function which gives a lot of control to the user. 

Let's break the options down into four categories:

1. Basic utilities
    - the semantic roles you're interested in (`roles_considered`)
    - where you would like to save the model (`output_path`)
    - whether you cant to track the function's progress (`progress_bar`)
    

2. Text preprocessing
    - basic text preprocessing steps 
    (`remove_punctuation`, `remove_digits`, remove `stop_words`, `stem` or `lemmatize` words, etc.)
    - would you like to replace verbs by their most common synonyms/antonyms? (`dimension_reduce_verbs`)


3. Named entities  
    - which semantic roles have named entities? (`roles_with_entities`)
    - how many named entities would you like to keep? (`top_n_entities`)

Technical details: under the hood, we work with SpaCy named entity recognizer to identify named entities. 
We consider tags related to places, organizations, people and events.


4. Unnamed entities (e.g., tax, government, dog, cat, etc.)
    - which semantic roles have unnamed entities? (`roles_with_embeddings`)
    - how many latent unnamed entities are there in the corpus? (`n_clusters`)

Technical details: under the hood, we embed semantic phrases without named entities and cluster them with 
K-Means. 

----------------------------

## Step 4: Get narrative statements based on the narrative model.

----------------------------

Once the narrative model is built, we can use it to extract narrative statements from any corpus (provided that the 
corpus is split into sentences and annotated for semantic roles). 

We call the function `get_narratives` for this purpose.

----------------------------

In [ ]:
from relatio.wrappers import get_narratives

final_statements = get_narratives(
    srl_res=srl_res,
    doc_index=split_sentences[0],  # doc names
    narrative_model=narrative_model,
    n_clusters=[0],  
    progress_bar=True,
)

In [ ]:
# The resulting pandas dataframe

print(final_statements.columns)

final_statements.head()

### Trying out different clustering scenarios

The choice of the number of clusters is corpus and application specific. Specifying a small number of latent entities leads to a large dimension reduction and may decrease entity coherence, whilst specifying a large number of latent entities may produce cluster redundancy. 

To help users try different clustering scenarios, the wrapper functions `build_narrative_model` and `get_narratives` allow users to experiment various clustering scenarios, which we detail below.

----------------------------

In `build_narrative_model`, the arguments `roles_with_embeddings` and `n_clusters` are specified as lists of lists arguments. This implies that you can cluster semantic roles separately (or together) and with different numbers of clusters. 

For example, a user could specify:
- `roles_with_embeddings = [["ARG0"],["ARG1"]]`
- `n_clusters = [[10,20],[10]]`
    
He/she would then cluster "ARG0" and "ARG1" separately and not consider "ARG2" for dimension reduction. For "ARG0", the clustering scenarios are a model with 10 and a model with 20 clusters. For "ARG1", the clustering scenario is a model with 10 clusters.


----------------------------

To extract narrative statements based on a clustering scenario, the `get_narratives` function also has the 
`n_clusters` argument. 

`n_clusters` is a list for the clustering scenarios. For instance, in our previous example, 
semantic roles "ARG0" and "ARG1" are clustered separately, so `n_clusters` expects two indices: one for the 
clustering scenario to pick for "ARG0" and one for the clustering scenario to pick for "ARG1".

For example, a user could specify:
- `n_clusters = [0,0]`

He/she would then extract narrative statements based on 10 clusters for "ARG0" and 10 clusters for "ARG1".

----------------------------

## Step 5: Model validation and basic analysis

----------------------------

The resulting `final_statments` object is a pandas dataframe which lists narrative statements found in documents and 
sentences of the corpus.

It is straight-forward to manually inspect the quality of the resulting entities and narrative statements.

----------------------------

In [ ]:
# Entity coherence
# Print most frequent phrases per entity

# Pool ARG0, ARG1 and ARG2 together

df1 = final_statements[['ARG0_lowdim', 'ARG0_highdim']]
df1.rename(columns={'ARG0_lowdim': 'ARG', 'ARG0_highdim': 'ARG-RAW'}, inplace=True)

df2 = final_statements[['ARG1_lowdim', 'ARG1_highdim']]
df2.rename(columns={'ARG1_lowdim': 'ARG', 'ARG1_highdim': 'ARG-RAW'}, inplace=True)

df3 = final_statements[['ARG2_lowdim', 'ARG2_highdim']]
df3.rename(columns={'ARG2_lowdim': 'ARG', 'ARG2_highdim': 'ARG-RAW'}, inplace=True)

df = df1.append(df2).reset_index(drop = True)
df = df.append(df3).reset_index(drop = True)

# Count semantic phrases

df = df.groupby(['ARG', 'ARG-RAW']).size().reset_index()
df.columns = ['ARG', 'ARG-RAW', 'count']

# Drop empty semantic phrases

df = df[df['ARG'] != ''] 

# Rearrange the data

df = df.groupby(['ARG']).apply(lambda x: x.sort_values(["count"], ascending = False))
df = df.reset_index(drop= True)
df = df.groupby(['ARG']).head(10)

df['ARG-RAW'] = df['ARG-RAW'] + ' - ' + df['count'].astype(str)
df['cluster_elements'] = df.groupby(['ARG'])['ARG-RAW'].transform(lambda x: ' | '.join(x))

df = df.drop_duplicates(subset=['ARG'])

df['cluster_elements'] = [', '.join(set(i.split(','))) for i in list(df['cluster_elements'])]

print('Entities to inspect:', len(df))

df = df[['ARG', 'cluster_elements']]

In [ ]:
for l in df.values.tolist():
    print('entity: \n %s \n' % [0])
    print('most frequent phrases: \n %s \n' % l[1])

In [ ]:
# Low-dimensional vs. high-dimensional narrative statements

# Replace negated verbs by "not-verb"

import numpy as np

final_statements['B-V_lowdim_with_neg'] = np.where(final_statements['B-ARGM-NEG_lowdim'] == True, 
                                          'not-' + final_statements['B-V_lowdim'], 
                                          final_statements['B-V_lowdim'])

final_statements['B-V_highdim_with_neg'] = np.where(final_statements['B-ARGM-NEG_highdim'] == True, 
                                           'not-' + final_statements['B-V_lowdim'], 
                                           final_statements['B-V_highdim'])

# Concatenate high-dimensional narratives (with text preprocessing but no clustering)

final_statements['narrative_highdim'] = (final_statements['ARG0_highdim'] + ' ' + 
                                         final_statements['B-V_highdim_with_neg'] + ' ' +  
                                         final_statements['ARG1_highdim'])

# Concatenate low-dimensional narratives (with clustering)

final_statements['narrative_lowdim'] = (final_statements['ARG0_lowdim'] + ' ' + 
                                        final_statements['B-V_highdim_with_neg'] + ' ' + 
                                        final_statements['ARG1_lowdim'])

# Focus on narratives with a ARG0-VERB-ARG1 structure (i.e. "complete narratives")

indexNames = final_statements[(final_statements['ARG0_lowdim'] == '')|
                             (final_statements['ARG1_lowdim'] == '')|
                             (final_statements['B-V_lowdim_with_neg'] == '')].index

complete_narratives = final_statements.drop(indexNames)

complete_narratives

In [ ]:
test = complete_narratives.merge(survey_df, how='left', left_on='doc', right_on='ResponseId')
test[test['Q7.5'].isin(['Always reject', 'reject'])]['narrative_lowdim'].value_counts()[:20]

In [ ]:
# Top high-dimensional complete narrative statements

complete_narratives['narrative_highdim'].value_counts().head(10)

In [ ]:
# Top low-dimensional complete narrative statements

complete_narratives['narrative_highdim'].value_counts().head(30)

In [ ]:
# Print ten random complete narratives with and without dimension reduction
#
# Specifying a small number of clusters leads to a large dimension reduction 
# and may decrease cluster coherence, whilst specifying a large number of clusters
# may produce cluster redundancy. 
#
# The choice of the number of clusters is corpus and application specific 
# (and was chosen at random in this notebook).

sample = complete_narratives.sample(10, random_state = 123).to_dict('records')

for d in sample:
    print('Original raw response: \n %s \n' %split_sentences[1][d['sentence']])
    print('High-dimensional narrative: \n %s \n' %d['narrative_highdim'])
    print('Low-dimensional narrative: \n %s \n' %d['narrative_lowdim'])
    print('--------------------------------------------------- \n')

In [ ]:
print(list(final_statements['ARG1_highdim']))

## Step 6: Visualization // Plotting narrative graphs
----------------------------

A collection of narrative statements has an intuitive network structure, in which the edges are verbs and the nodes are entities.

----------------------------

In [ ]:
temp = complete_narratives[["ARG0_lowdim", "ARG1_lowdim", "B-V_lowdim"]]
temp.columns = ["ARG0", "ARG1", "B-V"]
temp = temp[(temp["ARG0"] != "") & (temp["ARG1"] != "") & (temp["B-V"] != "")]
temp = temp.groupby(["ARG0", "ARG1", "B-V"]).size().reset_index(name="weight")
temp = temp.sort_values(by="weight", ascending=False)

In [ ]:
# Plot low-dimensional complete narrative statements in a directed multi-graph

from relatio.graphs import build_graph, draw_graph

temp = complete_narratives[["ARG0_lowdim", "ARG1_lowdim", "B-V_lowdim"]]
temp.columns = ["ARG0", "ARG1", "B-V"]
temp = temp[(temp["ARG0"] != "") & (temp["ARG1"] != "") & (temp["B-V"] != "")]
temp = temp.groupby(["ARG0", "ARG1", "B-V"]).size().reset_index(name="weight")
temp = temp.sort_values(by="weight", ascending=False).iloc[
    0:100
]  # pick top 100 most frequent narratives
temp = temp.to_dict(orient="records")

for l in temp:
    l["color"] = None

G = build_graph(
    dict_edges=temp, dict_args={}, edge_size=None, node_size=10, prune_network=False #rue
)

draw_graph(G, notebook=True, output_filename="all_data_relatio_result_nltk_stops_no_1_letter_filter_k50_all_named_ents_prune_network_false_top100narr.html")

In [ ]:
print(final_statements['ARG0_lowdim'].value_counts())
print(final_statements['ARG1_lowdim'].value_counts())

help(get_narratives)
#final_statements['ARG1_highdim']

In [ ]:
complete_narratives

In [ ]:
# Plot high-dimensional complete narrative statements in a directed multi-graph

from relatio.graphs import build_graph, draw_graph

temp = complete_narratives[["ARG0_highdim", "ARG1_highdim", "B-V_highdim"]]
temp.columns = ["ARG0", "ARG1", "B-V"]
temp = temp[(temp["ARG0"] != "") & (temp["ARG1"] != "") & (temp["B-V"] != "")]
temp = temp.groupby(["ARG0", "ARG1", "B-V"]).size().reset_index(name="weight")
temp = temp.sort_values(by="weight", ascending=False).iloc[
    0:200
]  # pick top 100 most frequent narratives
temp = temp.to_dict(orient="records")

for l in temp:
    l["color"] = None

G = build_graph(
    dict_edges=temp, dict_args={}, edge_size=None, node_size=10, prune_network=False #True
)

draw_graph(G, notebook=True, output_filename="all_data_relatio_result_nltk_stops_no_1_letter_filter_k50_all_named_ents_high_dim_prune_network_false_top200narr.html")

In [ ]:
# Plot low-dimensional complete narrative statements in a directed multi-graph

from relatio.graphs import build_graph, draw_graph

temp = final_statements[["ARG0_lowdim", "ARG1_lowdim", "B-V_lowdim"]]
temp.columns = ["ARG0", "ARG1", "B-V"]
temp = temp[(temp["ARG0"] != "") & (temp["ARG1"] != "") & (temp["B-V"] != "")]
temp = temp.groupby(["ARG0", "ARG1", "B-V"]).size().reset_index(name="weight")
temp = temp.sort_values(by="weight", ascending=False).iloc[
    0:100
]  # pick top 100 most frequent narratives
temp = temp.to_dict(orient="records")

for l in temp:
    l["color"] = None

G = build_graph(
    dict_edges=temp, dict_args={}, edge_size=None, node_size=10, prune_network=False #True
)

draw_graph(G, notebook=True, output_filename="final_statements_initial_relatio_result_nltk_stops_no_1_letter_filter_k50_low_dim_prune_false.html")

In [ ]:
# As a final comment, note that you can look up the specifics of any function with the help command.

help(build_narrative_model)